In [ ]:
!pip install rasterio

In [ ]:
import gdal
from osgeo import ogr
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import json
import rasterio
from shapely.geometry import LineString, Polygon, shape
import cv2
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = "/content/drive/My Drive/spacenet-data/"
image_path = drive_path + 'data/'
geojson_path = drive_pth + 'geojson_roads/'
mask_path = drive_Path + 'mask/'

In [ ]:
#installing aws cli
!curl "https://d1vvhvl2y92vvt.cloudfront.net/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
!unzip awscliv2.zip
!sudo ./aws/install

In [ ]:
#download tif from AWS
!/usr/local/bin/aws2 s3api get-object --bucket spacenet-dataset \
    --key spacenet/SN3_roads/train/AOI_2_Vegas/MS/SN3_roads_train_AOI_2_Vegas_MS_img100.tif \
    --request-payer requester SN3_roads_train_AOI_2_Vegas_PS-MS_img100.tif

In [ ]:
#download roads ge0json from AWS
!/usr/local/bin/aws2 s3api get-object --bucket spacenet-dataset \
    --key spacenet/SN3_roads/train/AOI_2_Vegas/geojson_roads/SN3_roads_train_AOI_2_Vegas_geojson_roads_img10.geojson \
    --request-payer requester SN3_roads_train_AOI_2_Vegas_geojson_roads_img10.geojson

In [ ]:
def create_mask_new(geojson_file, raster_src):

    srcRas_ds = gdal.Open(raster_src)
    cols = srcRas_ds.RasterXSize
    rows = srcRas_ds.RasterYSize

    mask = np.zeros((rows, cols))
    with open(geojson_file, 'r') as file:
    geo_json = json.load(file)
    with rasterio.open(raster_src) as map_image:
    for road in geo_json['features']:
        polys = list()
        geometry = road['geometry']
        if geometry['type'] == "MultiLineString":
            points = np.array(geometry['coordinates'][0])

        else:
            points = np.array(geometry['coordinates'])

        for point in points:
            coords = map_image.index(point[0], point[1])
            polys.append([coords[1], coords[0]])
            cv2.polylines(mask, np.int32([polys]), False, get_road_color(road_value), thickness = 2)


    mask = mask.astype(bool)
    return mask

In [ ]:
mask = create_mask_new('SN3_roads_train_AOI_2_Vegas_geojson_roads_img100.geojson', 'SN3_roads_train_AOI_2_Vegas_MS_img100.tif')
plt.imshow(mask)

In [ ]:
show_raster('SN3_roads_train_AOI_2_Vegas_MS_img100.tif')

In [ ]:
import pandas as pd
data = pd.read_csv('vegas_ms_list.txt', header = None)
image_list = data[3]
image_list = image_list.values

In [ ]:
missing = list()
for image_name in image_list[900:]:
    try:
        geojson_name = image_name.replace("MS", "geojson_roads").replace("tif", "geojson")
        mask = create_mask_new(geojson_path + geojson_name, image_path + image_name)
        mask_name = image_name.replace("MS", "mask").replace("tif", "npy")
        print(f"{count} {image_name} {geojson_name} {mask_name}")
        np.save(mask_path+mask_name, mask)
        count += 1
    except Exception:
        print("Error "+str(Exception))
        missing.append(image_name)
        pass

In [ ]:
for item in missing:
    print(item)

<h3>Improvements to be made:</h3><br>
    <h4>Include road type(which I guess can reveal info about road width) and speed limits into the mask.</h4>